In [198]:
import pandas as pd
import numpy as np

from time import time
from string import digits
import random
import threading

from geopy import Point
from geopy import Nominatim
from postal.parser import parse_address
from postal.expand import expand_address
import googlemaps
from googlemaps_api_key import key 

key = key.replace('†','')
gmaps = googlemaps.Client(key=key)
# https://console.cloud.google.com/apis/credentials?project=la-api-key


 API queries_quota: 60 



In [134]:
cd

/Users/karatechop


In [135]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)


In [155]:
itp_dusk = pd.read_csv('itp_dusk.csv')

In [156]:
la = itp_dusk[itp_dusk['city'] == 'Los Angeles'].reset_index(drop=True)
la['location'] = [i.lower().replace(' - los angeles','') for i in la['location']]
len(la['location'].unique())

60189

In [138]:
def first_char_NESW(street):
    
    try: 
        if street.split()[0] == 'n':
            street = 'north ' + ''.join(street[2:])
        elif street.split()[0] == 'e':
            street = 'east '  + ''.join(street[2:])
        elif street.split()[0] == 's':
            street = 'south ' + ''.join(street[2:])
        elif street.split()[0] == 'w':
            street = 'west ' + ''.join(street[2:])
    except:
        pass
    
    return street


In [132]:
sorted('west jefferson Boulevard & crenshaw Boulevard'.split('&'))

[' crenshaw Boulevard', 'west jefferson Boulevard ']

In [157]:
la['location'] = [i.lower().lstrip('').rstrip('') for i in la['location']]
la['location'] = la['location'].str.replace(' - Los Angeles','') 
la['location'] = la['location'].str.replace(' - los angeles','') 

la['location'] = la['location'].str.replace(' n ',' north ') 
la['location'] = la['location'].str.replace(' e ',' east ') 
la['location'] = la['location'].str.replace(' s ',' south ') 
la['location'] = la['location'].str.replace(' w ',' west ') 

la['location'] = la['location'].str.replace(' st ',' street ') 
la['location'] = la['location'].str.replace(' ave',' avenue')
la['location'] = la['location'].str.replace(' blvd',' boulevard')

la['location'] = [first_char_NESW(i) for i in la['location']]
la['location'] = [sorted(i.split('&'))[0] + ' & ' + sorted(i.split('&'))[1].lstrip().rstrip() if '&' in i 
                  else i.lstrip().rstrip() for i in la['location']]
len(la['location'].unique())

60177

In [280]:
freq = la['location'].value_counts()\
                     .rename_axis('location')\
                     .reset_index(name='count')

gmv3_priority = freq[(freq['location'].str.contains('&'))].reset_index(drop=True)
gmv3_array = higher_priority['location']
len(gmv3_array)

nom_priority = freq[~freq['location'].str.contains('&')].reset_index(drop=True)
nom_array = lower_priority['location']
len(nom_array)

12885

In [281]:
la_expanded = la
la_expanded['coords'] = None

nom_tracker = 0
gmv3_tracker = 0
failure_tracker = 0

In [283]:
def find_lat_lon_NOM():
    nom_start = time()
    counter = 0
    
    la_viewbox = ( (33.525861509262725,-117.32095033303995), (34.26915279876192,-118.52290082214722) )   
    la_bounds = gmaps.geocode('los angeles')[0]['geometry']['bounds']
    
    
    for address in nom_array: 
        
        try:
            geolocator = Nominatim(user_agent="kcharris@berkeley.edu")
            location = geolocator.geocode(address,viewbox=la_viewbox,bounded=True)

            coords = location.raw['lat'] + ' , ' + location.raw['lon']
            global nom_tracker
            nom_tracker += 1

        except:
            try:
                la_address = address + ' - los angeles'
                coords = str(gmaps.geocode(la_address,bounds=la_bounds)[0]['geometry']['location']['lat']) + ' , ' +\
                         str(gmaps.geocode(la_address,bounds=la_bounds)[0]['geometry']['location']['lng'])
                global gmv3_tracker
                gmv3_tracker += 1

            except:
                print(f'google maps and nominatim failed for {address}')
                coords = None
                global failure_tracker
                failure_tracker += 1
                pass
        
        la_expanded.loc[la_expanded['location']==address,'coords']=coords
        
        counter+=1
        if counter%1000==0:
            remaining = la_expanded['coords'].isna().sum()
            time_passed = round((time()-nom_start)/60,2)
            print(f'Counter is at {counter}. There are {remaining} coord values left. {time_passed} minutes have passed.') 

In [264]:
# # TEST
# find_lat_lon_NOM()
# print(nom_tracker,gmv3_tracker,failure_tracker)

Counter is at 10. There are 264099 coord values left. 0.17 minutes have passed.
13 2 0


In [284]:
def find_lat_lon_GMV3():
    gmv3_total_start = time()
    counter = 0
    
    la_bounds = gmaps.geocode('los angeles')[0]['geometry']['bounds']
    
    for address in gmv3_array: 
            
        try:
            la_address = address + ' - los angeles'
            coords = str(gmaps.geocode(la_address,bounds=la_bounds)[0]['geometry']['location']['lat']) + ' , ' +\
                     str(gmaps.geocode(la_address,bounds=la_bounds)[0]['geometry']['location']['lng'])
            
            global gmv3_tracker
            gmv3_tracker += 1

        except:
            print(f'ONLY google maps failed for {address}')
            coords = None
            
            global failure_tracker
            failure_tracker += 1
            pass
        
        la_expanded.loc[la_expanded['location']==address,'coords']=coords
        
        counter+=1
        if counter%5000==0:
            remaining = la_expanded['coords'].isna().sum()
            time_passed = round((time()-gmv3_total_start)/60,2)
            print(f'Counter is at {counter}. There are {remaining} coord values left. {time_passed} minutes have passed.') 

In [259]:
# # TEST
# find_lat_lon_GMV3()
# print(nom_tracker,gmv3_tracker,failure_tracker)

Counter is at 10. There are 266137 coord values left. 0.03 minutes have passed.
0 15 0


We thread both here. 

**FOLLOW UP:** Running this cost about $300 in google maps api credits. I've commented it out so it can't run again.

In [285]:
# if __name__ =="__main__":
#     t1 = threading.Thread(target=find_lat_lon_GMV3)
#     t2 = threading.Thread(target=find_lat_lon_NOM)
    
#     t1.start()
#     t2.start()
 
#     t1.join()
#     t2.join()
 
#     # both threads completely executed
#     print(f"\n\n\nBoth threads are done!\n\n{nom_tracker,gmv3_tracker,failure_tracker}")

Counter is at 5000. There are 124972 coord values left. 23.14 minutes have passed.
Counter is at 1000. There are 115973 coord values left. 27.1 minutes have passed.
Counter is at 10000. There are 85473 coord values left. 47.21 minutes have passed.
Counter is at 2000. There are 74286 coord values left. 56.85 minutes have passed.
Counter is at 15000. There are 61899 coord values left. 69.92 minutes have passed.
Counter is at 3000. There are 49655 coord values left. 85.51 minutes have passed.
Counter is at 20000. There are 45917 coord values left. 91.78 minutes have passed.
Counter is at 25000. There are 34585 coord values left. 113.34 minutes have passed.
Counter is at 4000. There are 32651 coord values left. 117.05 minutes have passed.
Counter is at 30000. There are 26217 coord values left. 135.57 minutes have passed.
Counter is at 5000. There are 22774 coord values left. 146.98 minutes have passed.
ONLY google maps failed for  85th st & central
Counter is at 35000. There are 19837 coor

In [297]:
la_expanded.loc[la_expanded['location']=='85th st & central','coords'] = '33.960799, -118.256405'
la_expanded.loc[la_expanded['location']=='1900 franklin','coords'] = '34.101071, -118.275197'

In [298]:
la_expanded.to_csv('la_lat_lon_expanded.csv',index=False)

In [299]:
test_df = pd.read_csv('la_lat_lon_expanded.csv')
test_df['coords'].value_counts()

34.0601079 , -118.2761598           893
33.9601319 , -118.2827255           716
34.0493961 , -118.2598891           627
34.0280561 , -118.2193901           574
34.22142669999999 , -118.4677605    544
                                   ... 
33.9544406 , -118.2782172             1
34.2084032 , -118.5273606             1
33.8102334 , -118.3085036             1
33.9891935 , -118.2140624             1
34.0402935 , -118.4633341             1
Name: coords, Length: 31264, dtype: int64